# Stock Signal Analysis Framework

## 1. Import Library

In [23]:
import dai
import time 
import random
import pandas as pd
import numpy as np
import bigcharts
from bigcharts import opts

## 2. Signal Settings

In [24]:
signal_param_dict = {
    "sql_signal":"""
        SELECT
            date,
            instrument,
            m_avg(close, 5)  AS ma_short,
            m_avg(close, 20) AS ma_long,
            IF(ma_short > ma_long AND m_lag(ma_short, 1) < m_lag(ma_long, 1), 1, 0) AS signal_buy,
            IF(ma_short < ma_long AND m_lag(ma_short, 1) > m_lag(ma_long, 1), 1, 0) AS signal_sell,
        FROM cn_stock_bar1d
    """,
    "sd": "2020-01-01",
    "ed": "2026-02-13",
    "sql_pool":"""
    WHERE instrument IN ('600519.SH')
    """
}

## 3. Signal Analysis

In [25]:
class SignalAnalysis:
    
    def __init__(self, signal_param_dict):
        self.sd = signal_param_dict["sd"]
        self.ed = signal_param_dict["ed"]

        self.sql_factor_process = self.get_sql_factor_process(signal_param_dict)
        self.df_signal = dai.query(self.sql_factor_process, filters = {"date":[self.sd, self.ed]}).df()

        self.df_buy_summary, self.df_sell_summary = self.summarize_overall(self.df_signal)
        self.df_buy_summary_year, self.df_sell_summary_year = self.summarize_by_year(self.df_signal)
        self.df_buy_summary_instrument, self.df_sell_summary_instrument = self.summarize_by_instrument(self.df_signal)

        self.df_buy_ret_daily, self.df_sell_ret_daily = self.plot_buy_sell_daily(self.df_signal)
        
    def get_sql_factor_process(self, signal_param_dict):
        
        sql_signal = signal_param_dict["sql_signal"]
        sql_pool   = signal_param_dict["sql_pool"]
        
        sql_factor_process = f"""
        WITH
        data_signal AS (
            {sql_signal}
            {sql_pool}
        ),
        data_processed AS (
            SELECT
                date,
                instrument,
                signal_buy,
                signal_sell,
                m_lead(open, 2)   / m_lead(open, 1) - 1 AS ret_001,
                m_lead(open, 3)   / m_lead(open, 1) - 1 AS ret_002,
                m_lead(open, 4)   / m_lead(open, 1) - 1 AS ret_003,
                m_lead(open, 5)   / m_lead(open, 1) - 1 AS ret_004,
                m_lead(open, 6)   / m_lead(open, 1) - 1 AS ret_005,
                m_lead(open, 11)  / m_lead(open, 1) - 1 AS ret_010,
                m_lead(open, 22)  / m_lead(open, 1) - 1 AS ret_021,
                m_lead(open, 43)  / m_lead(open, 1) - 1 AS ret_042,
                m_lead(open, 64)  / m_lead(open, 1) - 1 AS ret_063,
                m_lead(open, 127) / m_lead(open, 1) - 1 AS ret_126,
                m_lead(open, 253) / m_lead(open, 1) - 1 AS ret_252,
            FROM cn_stock_bar1d JOIN data_signal USING (date, instrument)
            QUALIFY (signal_buy = 1 OR signal_sell = 1)
        )
        SELECT *
        FROM data_processed
        """

        return sql_factor_process
    
    def summarize_overall(self, df):

        df = df.copy()
        df["date"] = pd.to_datetime(df["date"])

        ret_cols = [c for c in df.columns if c.startswith("ret_")]

        result_buy = {}
        result_sell = {}

        for c in ret_cols:

            n = int(c.replace("ret_", ""))

            df_buy = df[df["signal_buy"] == 1][["date", c]].dropna()

            if not df_buy.empty:
                buy_sum = df_buy[c].sum()
                buy_range = f"{df_buy['date'].min():%Y-%m-%d} ~ {df_buy['date'].max():%Y-%m-%d}"
            else:
                buy_sum = float("nan")
                buy_range = None

            result_buy[f"买入{n}日获得收益"] = buy_sum
            result_buy[f"买入{n}日策略时间"] = buy_range

            df_sell = df[df["signal_sell"] == 1][["date", c]].dropna()

            if not df_sell.empty:
                sell_sum = (-df_sell[c]).sum()
                sell_range = f"{df_sell['date'].min():%Y-%m-%d} ~ {df_sell['date'].max():%Y-%m-%d}"
            else:
                sell_sum = float("nan")
                sell_range = None

            result_sell[f"卖出{n}日避免损失"] = sell_sum
            result_sell[f"卖出{n}日策略时间"] = sell_range

        df_buy_summary  = pd.DataFrame([result_buy])
        df_sell_summary = pd.DataFrame([result_sell])

        bigcharts.Chart(
                data=df_buy_summary.round(4),
                type_="table",
                chart_options=dict(
                    title_opts=opts.ComponentTitleOpts(
                        title="信号整体收益\n信号（买入）整体收益"
                    )
                ),
                y=list(df_buy_summary.columns)
            ).render(display=True)
        
        bigcharts.Chart(
                data=df_sell_summary.round(4),
                type_="table",
                chart_options=dict(
                    title_opts=opts.ComponentTitleOpts(
                        title="信号（卖出）整体收益"
                    )
                ),
                y=list(df_sell_summary.columns)
            ).render(display=True)

        return df_buy_summary, df_sell_summary

    def summarize_by_year(self, df):

        df = df.copy()
        df["date"] = pd.to_datetime(df["date"])
        df["year"] = df["date"].dt.year

        ret_cols = [c for c in df.columns if c.startswith("ret_")]

        buy_rows = []
        sell_rows = []

        for year, sub_df in df.groupby("year"):

            result_buy = {"year": year}
            result_sell = {"year": year}

            for c in ret_cols:

                n = int(c.replace("ret_", ""))

                df_buy = sub_df[sub_df["signal_buy"] == 1][["date", c]].dropna()

                if not df_buy.empty:
                    buy_sum = df_buy[c].sum()
                    buy_range = f"{df_buy['date'].min():%Y-%m-%d} ~ {df_buy['date'].max():%Y-%m-%d}"
                else:
                    buy_sum = float("nan")
                    buy_range = None

                result_buy[f"买入{n}日获得收益"] = buy_sum
                result_buy[f"买入{n}日策略时间"] = buy_range

                df_sell = sub_df[sub_df["signal_sell"] == 1][["date", c]].dropna()

                if not df_sell.empty:
                    sell_sum = (-df_sell[c]).sum()
                    sell_range = f"{df_sell['date'].min():%Y-%m-%d} ~ {df_sell['date'].max():%Y-%m-%d}"
                else:
                    sell_sum = float("nan")
                    sell_range = None

                result_sell[f"卖出{n}日避免损失"] = sell_sum
                result_sell[f"卖出{n}日策略时间"] = sell_range

            buy_rows.append(result_buy)
            sell_rows.append(result_sell)

        df_buy_summary  = pd.DataFrame(buy_rows).sort_values("year").reset_index(drop=True)
        df_sell_summary = pd.DataFrame(sell_rows).sort_values("year").reset_index(drop=True)

        bigcharts.Chart(
                data=df_buy_summary.round(4),
                type_="table",
                chart_options=dict(
                    title_opts=opts.ComponentTitleOpts(
                        title="信号年度收益\n信号（买入）年度收益"
                    )
                ),
                y=list(df_buy_summary.columns)
            ).render(display=True)
        
        bigcharts.Chart(
                data=df_sell_summary.round(4),
                type_="table",
                chart_options=dict(
                    title_opts=opts.ComponentTitleOpts(
                        title="信号（卖出）年度收益"
                    )
                ),
                y=list(df_sell_summary.columns)
            ).render(display=True)

        return df_buy_summary, df_sell_summary

    def summarize_by_instrument(self, df):

        df = df.copy()
        df["date"] = pd.to_datetime(df["date"])

        ret_cols = [c for c in df.columns if c.startswith("ret_")]

        buy_rows = []
        sell_rows = []

        for instrument, sub_df in df.groupby("instrument"):

            result_buy = {"instrument": instrument}
            result_sell = {"instrument": instrument}

            for c in ret_cols:

                n = int(c.replace("ret_", ""))

                df_buy = sub_df[sub_df["signal_buy"] == 1][["date", c]].dropna()

                if not df_buy.empty:
                    buy_sum = df_buy[c].sum()
                    buy_range = f"{df_buy['date'].min():%Y-%m-%d} ~ {df_buy['date'].max():%Y-%m-%d}"
                else:
                    buy_sum = float("nan")
                    buy_range = None

                result_buy[f"买入{n}日获得收益"] = buy_sum
                result_buy[f"买入{n}日策略时间"] = buy_range

                df_sell = sub_df[sub_df["signal_sell"] == 1][["date", c]].dropna()

                if not df_sell.empty:
                    sell_sum = (-df_sell[c]).sum()
                    sell_range = f"{df_sell['date'].min():%Y-%m-%d} ~ {df_sell['date'].max():%Y-%m-%d}"
                else:
                    sell_sum = float("nan")
                    sell_range = None

                result_sell[f"卖出{n}日避免损失"] = sell_sum
                result_sell[f"卖出{n}日策略时间"] = sell_range

            buy_rows.append(result_buy)
            sell_rows.append(result_sell)

        df_buy_summary  = pd.DataFrame(buy_rows)
        df_sell_summary = pd.DataFrame(sell_rows)

        bigcharts.Chart(
                data=df_buy_summary.round(4),
                type_="table",
                chart_options=dict(
                    title_opts=opts.ComponentTitleOpts(
                        title="信号标的收益\n信号（买入）标的收益"
                    )
                ),
                y=list(df_buy_summary.columns)
            ).render(display=True)
        
        bigcharts.Chart(
                data=df_sell_summary.round(4),
                type_="table",
                chart_options=dict(
                    title_opts=opts.ComponentTitleOpts(
                        title="信号（卖出）标的收益"
                    )
                ),
                y=list(df_sell_summary.columns)
            ).render(display=True)

        return df_buy_summary, df_sell_summary

    def plot_buy_sell_daily(self, df):

        df = df.copy()
        df["date"] = pd.to_datetime(df["date"])
        df = df.sort_values("date")

        ret_cols = sorted(
            [c for c in df.columns if c.startswith("ret_")],
            key=lambda x: int(x.replace("ret_", ""))
        )

        def gen_gradient(n):
            orange = np.array([255, 165, 0])
            red    = np.array([220, 0, 0])
            purple = np.array([128, 0, 128])

            colors = []
            for i in range(n):
                t = i / (n - 1) if n > 1 else 0.0
                if t < 0.5:
                    r = t / 0.5
                    rgb = orange * (1 - r) + red * r
                else:
                    r = (t - 0.5) / 0.5
                    rgb = red * (1 - r) + purple * r
                colors.append(f"rgb({int(rgb[0])},{int(rgb[1])},{int(rgb[2])})")
            return colors

        colors = gen_gradient(len(ret_cols))

        label_map = {c: f"{int(c.replace('ret_', ''))}日" for c in ret_cols}
        y_cols = [label_map[c] for c in ret_cols]  # 画图用的新列名顺序

        series_options = {}
        for col_label, color in zip(y_cols, colors):
            series_options[col_label] = dict(
                color=color,
                is_symbol_show=False,
                linestyle_opts=opts.LineStyleOpts(color=color, width=2),
            )

        buy = df[df["signal_buy"] == 1][["date"] + ret_cols].copy()
        buy = buy.sort_values("date")
        buy[ret_cols] = buy[ret_cols].fillna(0).cumsum()
        buy = buy.rename(columns=label_map)
        buy["date_str"] = buy["date"].dt.strftime("%Y-%m-%d")

        bigcharts.Chart(
            data=buy.round(6),
            type_="line",
            x="date_str",
            y=y_cols,
            chart_options=dict(
                legend_opts=opts.LegendOpts(pos_top="bottom"),
                title_opts=opts.TitleOpts(title="""信号（买入）累计收益曲线"""),
                xaxis_opts=opts.AxisOpts(split_number=8),
            ),
            series_options=series_options,
        ).render(display=True)

        sell = df[df["signal_sell"] == 1][["date"] + ret_cols].copy()
        sell = sell.sort_values("date")
        sell[ret_cols] = (-sell[ret_cols]).fillna(0).cumsum()
        sell = sell.rename(columns=label_map)
        sell["date_str"] = sell["date"].dt.strftime("%Y-%m-%d")

        bigcharts.Chart(
            data=sell.round(6),
            type_="line",
            x="date_str",
            y=y_cols,
            chart_options=dict(
                legend_opts=opts.LegendOpts(pos_top="bottom"),
                title_opts=opts.TitleOpts(title="信号（卖出）累计收益曲线"),
                xaxis_opts=opts.AxisOpts(split_number=8),
            ),
            series_options=series_options,
        ).render(display=True)

        return buy, sell


In [26]:
signal_analysis = SignalAnalysis(signal_param_dict)

买入1日获得收益,买入1日策略时间,买入2日获得收益,买入2日策略时间,买入3日获得收益,买入3日策略时间,买入4日获得收益,买入4日策略时间,买入5日获得收益,买入5日策略时间,买入10日获得收益,买入10日策略时间,买入21日获得收益,买入21日策略时间,买入42日获得收益,买入42日策略时间,买入63日获得收益,买入63日策略时间,买入126日获得收益,买入126日策略时间,买入252日获得收益,买入252日策略时间
0.1171,2020-02-13 ~ 2026-02-03,-0.0706,2020-02-13 ~ 2026-02-03,-0.3203,2020-02-13 ~ 2026-02-03,-0.2487,2020-02-13 ~ 2026-02-03,-0.2628,2020-02-13 ~ 2026-02-03,-0.5237,2020-02-13 ~ 2025-12-22,-0.3973,2020-02-13 ~ 2025-12-22,0.0117,2020-02-13 ~ 2025-11-12,0.9674,2020-02-13 ~ 2025-11-12,0.9429,2020-02-13 ~ 2025-07-07,1.032,2020-02-13 ~ 2024-12-12


卖出1日避免损失,卖出1日策略时间,卖出2日避免损失,卖出2日策略时间,卖出3日避免损失,卖出3日策略时间,卖出4日避免损失,卖出4日策略时间,卖出5日避免损失,卖出5日策略时间,卖出10日避免损失,卖出10日策略时间,卖出21日避免损失,卖出21日策略时间,卖出42日避免损失,卖出42日策略时间,卖出63日避免损失,卖出63日策略时间,卖出126日避免损失,卖出126日策略时间,卖出252日避免损失,卖出252日策略时间
0.0912,2020-02-28 ~ 2026-01-15,0.1558,2020-02-28 ~ 2026-01-15,0.0863,2020-02-28 ~ 2026-01-15,0.2266,2020-02-28 ~ 2026-01-15,0.2142,2020-02-28 ~ 2026-01-15,0.2066,2020-02-28 ~ 2026-01-15,0.2593,2020-02-28 ~ 2025-11-27,-0.0594,2020-02-28 ~ 2025-11-27,0.049,2020-02-28 ~ 2025-10-29,-1.0117,2020-02-28 ~ 2025-08-01,-0.9114,2020-02-28 ~ 2025-01-02


year,买入1日获得收益,买入1日策略时间,买入2日获得收益,买入2日策略时间,买入3日获得收益,买入3日策略时间,买入4日获得收益,买入4日策略时间,买入5日获得收益,买入5日策略时间,买入10日获得收益,买入10日策略时间,买入21日获得收益,买入21日策略时间,买入42日获得收益,买入42日策略时间,买入63日获得收益,买入63日策略时间,买入126日获得收益,买入126日策略时间,买入252日获得收益,买入252日策略时间
2020,-0.0149,2020-02-13 ~ 2020-11-09,-0.032,2020-02-13 ~ 2020-11-09,-0.0564,2020-02-13 ~ 2020-11-09,-0.0432,2020-02-13 ~ 2020-11-09,0.018,2020-02-13 ~ 2020-11-09,-0.0587,2020-02-13 ~ 2020-11-09,0.1821,2020-02-13 ~ 2020-11-09,0.7732,2020-02-13 ~ 2020-11-09,1.6977,2020-02-13 ~ 2020-11-09,2.5246,2020-02-13 ~ 2020-11-09,2.7525,2020-02-13 ~ 2020-11-09
2021,0.016,2021-03-31 ~ 2021-11-22,0.1062,2021-03-31 ~ 2021-11-22,0.103,2021-03-31 ~ 2021-11-22,0.1646,2021-03-31 ~ 2021-11-22,0.1502,2021-03-31 ~ 2021-11-22,0.0714,2021-03-31 ~ 2021-11-22,0.2241,2021-03-31 ~ 2021-11-22,0.1849,2021-03-31 ~ 2021-11-22,-0.0022,2021-03-31 ~ 2021-11-22,-0.2498,2021-03-31 ~ 2021-11-22,-0.3597,2021-03-31 ~ 2021-11-22
2022,-0.0292,2022-01-27 ~ 2022-11-15,-0.1557,2022-01-27 ~ 2022-11-15,-0.2373,2022-01-27 ~ 2022-11-15,-0.1977,2022-01-27 ~ 2022-11-15,-0.2024,2022-01-27 ~ 2022-11-15,-0.1908,2022-01-27 ~ 2022-11-15,-0.2843,2022-01-27 ~ 2022-11-15,-0.1809,2022-01-27 ~ 2022-11-15,0.0024,2022-01-27 ~ 2022-11-15,-0.2986,2022-01-27 ~ 2022-11-15,-0.0501,2022-01-27 ~ 2022-11-15
2023,0.0079,2023-02-20 ~ 2023-12-28,-0.0168,2023-02-20 ~ 2023-12-28,-0.0628,2023-02-20 ~ 2023-12-28,-0.1083,2023-02-20 ~ 2023-12-28,-0.1156,2023-02-20 ~ 2023-12-28,-0.1147,2023-02-20 ~ 2023-12-28,-0.1583,2023-02-20 ~ 2023-12-28,-0.1734,2023-02-20 ~ 2023-12-28,-0.2321,2023-02-20 ~ 2023-12-28,-0.3439,2023-02-20 ~ 2023-12-28,-0.9282,2023-02-20 ~ 2023-12-28
2024,0.0806,2024-02-07 ~ 2024-12-12,-0.0595,2024-02-07 ~ 2024-12-12,-0.1498,2024-02-07 ~ 2024-12-12,-0.1394,2024-02-07 ~ 2024-12-12,-0.166,2024-02-07 ~ 2024-12-12,-0.2863,2024-02-07 ~ 2024-12-12,-0.3816,2024-02-07 ~ 2024-12-12,-0.4848,2024-02-07 ~ 2024-12-12,-0.4681,2024-02-07 ~ 2024-12-12,-0.5894,2024-02-07 ~ 2024-12-12,-0.3824,2024-02-07 ~ 2024-12-12
2025,0.0332,2025-02-14 ~ 2025-12-22,0.0401,2025-02-14 ~ 2025-12-22,0.0607,2025-02-14 ~ 2025-12-22,0.0483,2025-02-14 ~ 2025-12-22,0.0396,2025-02-14 ~ 2025-12-22,0.0554,2025-02-14 ~ 2025-12-22,0.0206,2025-02-14 ~ 2025-12-22,-0.1072,2025-02-14 ~ 2025-11-12,-0.0303,2025-02-14 ~ 2025-11-12,-0.1,2025-02-14 ~ 2025-07-07,nan,None
2026,0.0236,2026-02-03 ~ 2026-02-03,0.0471,2026-02-03 ~ 2026-02-03,0.0222,2026-02-03 ~ 2026-02-03,0.0269,2026-02-03 ~ 2026-02-03,0.0133,2026-02-03 ~ 2026-02-03,nan,None,nan,None,nan,None,nan,None,nan,None,nan,None


year,卖出1日避免损失,卖出1日策略时间,卖出2日避免损失,卖出2日策略时间,卖出3日避免损失,卖出3日策略时间,卖出4日避免损失,卖出4日策略时间,卖出5日避免损失,卖出5日策略时间,卖出10日避免损失,卖出10日策略时间,卖出21日避免损失,卖出21日策略时间,卖出42日避免损失,卖出42日策略时间,卖出63日避免损失,卖出63日策略时间,卖出126日避免损失,卖出126日策略时间,卖出252日避免损失,卖出252日策略时间
2020,-0.0547,2020-02-28 ~ 2020-10-28,-0.0922,2020-02-28 ~ 2020-10-28,-0.0846,2020-02-28 ~ 2020-10-28,-0.1185,2020-02-28 ~ 2020-10-28,-0.1134,2020-02-28 ~ 2020-10-28,-0.1237,2020-02-28 ~ 2020-10-28,-0.2596,2020-02-28 ~ 2020-10-28,-0.6555,2020-02-28 ~ 2020-10-28,-1.1894,2020-02-28 ~ 2020-10-28,-2.2275,2020-02-28 ~ 2020-10-28,-2.2107,2020-02-28 ~ 2020-10-28
2021,0.0394,2021-02-26 ~ 2021-10-29,0.1163,2021-02-26 ~ 2021-10-29,0.1131,2021-02-26 ~ 2021-10-29,0.1216,2021-02-26 ~ 2021-10-29,0.0496,2021-02-26 ~ 2021-10-29,0.0422,2021-02-26 ~ 2021-10-29,-0.0925,2021-02-26 ~ 2021-10-29,-0.0106,2021-02-26 ~ 2021-10-29,0.246,2021-02-26 ~ 2021-10-29,0.4806,2021-02-26 ~ 2021-10-29,0.5613,2021-02-26 ~ 2021-10-29
2022,0.0477,2022-01-04 ~ 2022-10-11,0.0861,2022-01-04 ~ 2022-10-11,0.05,2022-01-04 ~ 2022-10-11,0.0914,2022-01-04 ~ 2022-10-11,0.1463,2022-01-04 ~ 2022-10-11,0.3609,2022-01-04 ~ 2022-10-11,0.6121,2022-01-04 ~ 2022-10-11,0.4357,2022-01-04 ~ 2022-10-11,0.4058,2022-01-04 ~ 2022-10-11,0.3433,2022-01-04 ~ 2022-10-11,0.1652,2022-01-04 ~ 2022-10-11
2023,-0.0453,2023-02-06 ~ 2023-11-28,-0.0229,2023-02-06 ~ 2023-11-28,-0.0569,2023-02-06 ~ 2023-11-28,-0.0383,2023-02-06 ~ 2023-11-28,-0.0039,2023-02-06 ~ 2023-11-28,-0.1135,2023-02-06 ~ 2023-11-28,0.0943,2023-02-06 ~ 2023-11-28,0.1012,2023-02-06 ~ 2023-11-28,0.1025,2023-02-06 ~ 2023-11-28,0.1065,2023-02-06 ~ 2023-11-28,0.3631,2023-02-06 ~ 2023-11-28
2024,0.0488,2024-01-09 ~ 2024-11-19,0.0133,2024-01-09 ~ 2024-11-19,-0.0001,2024-01-09 ~ 2024-11-19,0.0851,2024-01-09 ~ 2024-11-19,0.0821,2024-01-09 ~ 2024-11-19,0.0515,2024-01-09 ~ 2024-11-19,-0.1638,2024-01-09 ~ 2024-11-19,-0.0498,2024-01-09 ~ 2024-11-19,0.351,2024-01-09 ~ 2024-11-19,0.2038,2024-01-09 ~ 2024-11-19,0.1684,2024-01-09 ~ 2024-11-19
2025,0.0459,2025-01-02 ~ 2025-11-27,0.044,2025-01-02 ~ 2025-11-27,0.0495,2025-01-02 ~ 2025-11-27,0.0544,2025-01-02 ~ 2025-11-27,0.0141,2025-01-02 ~ 2025-11-27,0.0179,2025-01-02 ~ 2025-11-27,0.0688,2025-01-02 ~ 2025-11-27,0.1195,2025-01-02 ~ 2025-11-27,0.133,2025-01-02 ~ 2025-10-29,0.0815,2025-01-02 ~ 2025-08-01,0.0412,2025-01-02 ~ 2025-01-02
2026,0.0094,2026-01-15 ~ 2026-01-15,0.0111,2026-01-15 ~ 2026-01-15,0.0154,2026-01-15 ~ 2026-01-15,0.0308,2026-01-15 ~ 2026-01-15,0.0394,2026-01-15 ~ 2026-01-15,-0.0287,2026-01-15 ~ 2026-01-15,nan,None,nan,None,nan,None,nan,None,nan,None


instrument,买入1日获得收益,买入1日策略时间,买入2日获得收益,买入2日策略时间,买入3日获得收益,买入3日策略时间,买入4日获得收益,买入4日策略时间,买入5日获得收益,买入5日策略时间,买入10日获得收益,买入10日策略时间,买入21日获得收益,买入21日策略时间,买入42日获得收益,买入42日策略时间,买入63日获得收益,买入63日策略时间,买入126日获得收益,买入126日策略时间,买入252日获得收益,买入252日策略时间
600519.SH,0.1171,2020-02-13 ~ 2026-02-03,-0.0706,2020-02-13 ~ 2026-02-03,-0.3203,2020-02-13 ~ 2026-02-03,-0.2487,2020-02-13 ~ 2026-02-03,-0.2628,2020-02-13 ~ 2026-02-03,-0.5237,2020-02-13 ~ 2025-12-22,-0.3973,2020-02-13 ~ 2025-12-22,0.0117,2020-02-13 ~ 2025-11-12,0.9674,2020-02-13 ~ 2025-11-12,0.9429,2020-02-13 ~ 2025-07-07,1.032,2020-02-13 ~ 2024-12-12


instrument,卖出1日避免损失,卖出1日策略时间,卖出2日避免损失,卖出2日策略时间,卖出3日避免损失,卖出3日策略时间,卖出4日避免损失,卖出4日策略时间,卖出5日避免损失,卖出5日策略时间,卖出10日避免损失,卖出10日策略时间,卖出21日避免损失,卖出21日策略时间,卖出42日避免损失,卖出42日策略时间,卖出63日避免损失,卖出63日策略时间,卖出126日避免损失,卖出126日策略时间,卖出252日避免损失,卖出252日策略时间
600519.SH,0.0912,2020-02-28 ~ 2026-01-15,0.1558,2020-02-28 ~ 2026-01-15,0.0863,2020-02-28 ~ 2026-01-15,0.2266,2020-02-28 ~ 2026-01-15,0.2142,2020-02-28 ~ 2026-01-15,0.2066,2020-02-28 ~ 2026-01-15,0.2593,2020-02-28 ~ 2025-11-27,-0.0594,2020-02-28 ~ 2025-11-27,0.049,2020-02-28 ~ 2025-10-29,-1.0117,2020-02-28 ~ 2025-08-01,-0.9114,2020-02-28 ~ 2025-01-02
